In [173]:
import os
import sys

import numpy as np
import pandas as pd

import sklearn

In [174]:
demographic = pd.read_csv('raw_data/demographic.csv', encoding='ISO-8859-1')
diet = pd.read_csv('raw_data/diet.csv', encoding='ISO-8859-1')
examination = pd.read_csv('raw_data/examination.csv', encoding='ISO-8859-1')
labs = pd.read_csv('raw_data/labs.csv', encoding='ISO-8859-1')
medication = pd.read_csv('raw_data/medications.csv', encoding='ISO-8859-1')
questionnaire = pd.read_csv('raw_data/questionnaire.csv', encoding='ISO-8859-1')

# sex, race, age(age group)
# health outcome (examination, labs)
# lifestyle (diet, questionnaire)
variables = pd.read_csv('variables.csv')
variables_lst = variables['Variable'].tolist()

In [175]:
# select columns in each dataset that are in the variables list (may not in it), join then by SEQN in each dataset

demographic_subset = demographic[["SEQN"]+list(set(variables_lst).intersection(demographic.columns))]
diet_subset = diet[["SEQN"]+list(set(variables_lst).intersection(diet.columns))]
examination_subset = examination[["SEQN"]+list(set(variables_lst).intersection(examination.columns))]
labs_subset = labs[["SEQN"]+list(set(variables_lst).intersection(labs.columns))]
medication_subset = medication[["SEQN"]+list(set(variables_lst).intersection(medication.columns))]
questionnaire_subset = questionnaire[["SEQN"]+list(set(variables_lst).intersection(questionnaire.columns))]
# merge all datasets
merged_df = demographic_subset.merge(diet_subset, on='SEQN').merge(examination_subset, on='SEQN').merge(labs_subset, on='SEQN').merge(medication_subset, on='SEQN').merge(questionnaire_subset, on='SEQN').drop_duplicates()

# replace all categorical variables with the categroies value in the Notes column in the variables.csv
# for example: DMDEDUC2,Education Level (Adults 20+),Demographics,Categorical,"Less than 9th grade, 9–11th grade, High school/GED, Some college/AA degree, College graduate or above"
# then 1 is "Less than 9th grade", 2 is "9–11th grade" and so on
# categorical variables
categorical_variables = variables[variables['Data Type'] == 'Categorical']
for i in range(1, len(merged_df.columns)):
    if merged_df.columns[i] in categorical_variables['Variable'].tolist():
        categories = categorical_variables[categorical_variables['Variable'] == merged_df.columns[i]]['Notes'].values[0].split(', ')
        # convert any value out of range to be NaN
        merged_df[merged_df.columns[i]] = merged_df[merged_df.columns[i]].apply(lambda x: categories[int(x)-1] if not pd.isnull(x) and int(x) <= len(categories) else np.nan)

# replace column names each column names' corresponding Description
merged_df.columns = ["Respondent"] + variables.set_index('Variable').loc[merged_df.columns[1:]]['Description'].tolist()  
merged_df

,Respondent,Examination Month Period,Household Size,Age of the Participant (years),Marital Status,Education Level (Adults 20+),Gender of the Participant,Pregnancy Status at Examination,Country of Birth,Race/Ethnicity,...,Platelet Count (10^3 cells/uL),Hemoglobin (g/dL),Engaged in Vigorous Activity in Past 30 Days,Eternal Alcohol Consumption Status (ever had at least 12 drinks in life),Smoked at Least 100 Cigarettes in Life,Sleep Duration (hours),Current Smoking Frequency,Time Spent Watching TV or Videos (minutes/day),Frequency of Vigorous Physical Activity per Week,Frequency of Muscle-Strengthening Activities per Week
0,73557,November 1–April 30,3,69,Separated,High school/GED,Male,NaN,Born in 50 U.S. states or D.C.,Non-Hispanic Black,...,204.0,15.2,No,NaN,Yes,7.0,Not at all,2.0,2.0,8.0
2,73558,November 1–April 30,4,54,Married,High school/GED,Male,NaN,Born in 50 U.S. states or D.C.,Non-Hispanic White,...,314.0,11.9,No,NaN,Yes,9.0,Some days,2.0,4.0,8.0
6,73559,May 1–October 31,2,72,Married,Some college/AA degree,Male,NaN,Born in 50 U.S. states or D.C.,Non-Hispanic White,...,237.0,17.2,No,NaN,Yes,8.0,Not at all,2.0,4.0,0.0
11,73560,November 1–April 30,4,9,NaN,NaN,Male,NaN,Born in 50 U.S. states or D.C.,Non-Hispanic White,...,240.0,12.9,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0
12,73561,November 1–April 30,2,73,Married,College graduate or above,Female,NaN,Born in 50 U.S. states or D.C.,Non-Hispanic White,...,300.0,14.5,No,NaN,No,9.0,NaN,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19575,83727,May 1–October 31,5,26,Never married,College graduate or above,Male,NaN,Born in 50 U.S. states or D.C.,Other Hispanic,...,226.0,14.9,No,NaN,No,7.0,NaN,2.0,2.0,1.0
19576,83728,May 1–October 31,4,2,NaN,NaN,Female,NaN,Born in 50 U.S. states or D.C.,Mexican American,...,441.0,12.2,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0
19577,83729,May 1–October 31,1,42,Divorced,College graduate or above,Female,Not pregnant,Born in Mexico,Non-Hispanic Black,...,213.0,12.2,No,NaN,No,7.0,NaN,2.0,5.0,1.0
19578,83730,November 1–April 30,4,7,NaN,NaN,Male,NaN,Born in 50 U.S. states or D.C.,Other Hispanic,...,352.0,13.7,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0


In [176]:
# check any col have nan > 50%
nan_count = merged_df.isna().sum()
nan_count = nan_count[nan_count > 0]
nan_count = nan_count/merged_df.shape[0]
nan_count = nan_count[nan_count > 0.5]
nan_count

Pregnancy Status at Examination                                             0.876185
Urine Creatinine (mg/dL)                                                    0.725874
Calculated LDL Cholesterol (mg/dL)                                          0.683583
Triglycerides (mg/dL)                                                       0.679405
Eternal Alcohol Consumption Status (ever had at least 12 drinks in life)    0.834200
Current Smoking Frequency                                                   0.746255
dtype: float64

In [177]:
# save processed data
merged_df.to_csv('processed_data.csv', index=False)